In [1]:
import cv2
import mediapipe as mp
import pybullet as p
import time

# Initialize MediaPipe Hand module
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()

# Disconnect from the existing PyBullet GUI connection if it exists
try:
    p.disconnect()
except:
    pass

# Connect to the PyBullet physics server with a new GUI connection
physicsClient = p.connect(p.GUI)

# Set gravity in the simulation
p.setGravity(0, 0, -9.8)

# Load a simple hand model into PyBullet
hand_urdf_path = r'd://urdf_files/hand/urdf/shadow_hand.urdf'  # Replace with the actual path
hand_id = p.loadURDF(hand_urdf_path, useFixedBase=True)

# Open webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the BGR image to RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the frame with MediaPipe Hands
    results = hands.process(rgb_frame)

    # Check if hands are detected
    if results.multi_hand_landmarks:
        hand_landmarks = results.multi_hand_landmarks[0]

        # Get the coordinates of the tip of the index finger (landmark index 8)
        index_finger_tip = hand_landmarks.landmark[8]
        index_finger_x, index_finger_y = index_finger_tip.x, index_finger_tip.y

        # Update PyBullet hand position based on the detected coordinates
        p.resetBasePositionAndOrientation(hand_id, [index_finger_x, index_finger_y, 1.0], [0, 0, 0, 1])

    # Step the PyBullet simulation
    p.stepSimulation()

    # Render the simulation
    p.getCameraImage(800, 600)

    # Display the frame
    cv2.imshow('Hand Gesture Simulation', frame)

    # Break the loop when 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Disconnect from the PyBullet physics server
p.disconnect()

# Release the webcam and close all windows
cap.release()
cv2.destroyAllWindows()


d:\ProgramData\anaconda3\envs\tf-gpu\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
d:\ProgramData\anaconda3\envs\tf-gpu\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
d:\ProgramData\anaconda3\envs\tf-gpu\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [3]:
import cv2
import mediapipe as mp
import pybullet as p
import time
import numpy as np

# Initialize MediaPipe Hand module
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()

# Disconnect from the existing PyBullet GUI connection if it exists
try:
    p.disconnect()
except:
    pass

# Connect to the PyBullet physics server with a new GUI connection
physicsClient = p.connect(p.GUI)

# Set gravity in the simulation
p.setGravity(0, 0, -9.8)

# Load a simple hand model into PyBullet
hand_urdf_path = "d://urdf_files/hand/urdf/shadow_hand.urdf"  # Replace with the actual path
hand_id = p.loadURDF(hand_urdf_path, useFixedBase=True)

# Open webcam
cap = cv2.VideoCapture(0)

# Finger joint indices in PyBullet (adjust these based on your URDF file)
finger_joint_indices = [0, 1, 2, 3, 4,6,7,8,9,10]  # Adjust based on your URDF file

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the BGR image to RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the frame with MediaPipe Hands
    results = hands.process(rgb_frame)

    # Check if hands are detected
    if results.multi_hand_landmarks:
        hand_landmarks = results.multi_hand_landmarks[0]

        # Control forearm-wrist joint based on the detected coordinates
        wrist_joint_angle = np.interp(hand_landmarks.landmark[8].y, [0, 1], [-0.5236, 0.1745])
        p.setJointMotorControl2(hand_id, jointIndex=0, controlMode=p.POSITION_CONTROL, targetPosition=wrist_joint_angle)

        # Control finger joints based on the detected coordinates
        for i, joint_index in enumerate(finger_joint_indices):
            finger_joint_angle = np.interp(hand_landmarks.landmark[i + 2].y, [0, 1], [-0.5236, 0.1745])
            p.setJointMotorControl2(hand_id, jointIndex=joint_index, controlMode=p.POSITION_CONTROL, targetPosition=finger_joint_angle)

    # Step the PyBullet simulation
    p.stepSimulation()

    # Render the simulation
    p.getCameraImage(800, 600)

    # Display the frame
    cv2.imshow('Hand Gesture Simulation', frame)

    # Break the loop when 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Disconnect from the PyBullet physics server
p.disconnect()

# Release the webcam and close all windows
cap.release()
cv2.destroyAllWindows()


In [6]:
import cv2
import mediapipe as mp
import pybullet as p
import time
import numpy as np

# Initialize MediaPipe Hand module
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()

# Disconnect from the existing PyBullet GUI connection if it exists
try:
    p.disconnect()
except:
    pass

# Connect to the PyBullet physics server with a new GUI connection
physicsClient = p.connect(p.GUI)

# Set gravity in the simulation
p.setGravity(0, 0, -9.8)

# Load a simple hand model into PyBullet
hand_urdf_path = "d://urdf_files/hand/urdf/shadow_hand.urdf"  # Replace with the actual path
hand_id = p.loadURDF(hand_urdf_path, useFixedBase=True)

# Open webcam
cap = cv2.VideoCapture(0)

# Define a mapping between joint names and corresponding MediaPipe landmarks
joint_to_landmark_mapping = {
    "thumb_joint1": 2,  # Replace with the actual landmark index for thumb_joint1
    "thumb_joint2": 3,  # Replace with the actual landmark index for thumb_joint2
    "thumb_joint3": 4   # Replace with the actual landmark index for thumb_joint3
}

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the BGR image to RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the frame with MediaPipe Hands
    results = hands.process(rgb_frame)

    # Check if hands are detected
    if results.multi_hand_landmarks:
        hand_landmarks = results.multi_hand_landmarks[0]

        # Control thumb joints based on the detected coordinates
        for joint_name, landmark_index in joint_to_landmark_mapping.items():
            joint_angle = np.interp(hand_landmarks.landmark[landmark_index].y, [0, 1], [-1.047, 1.047])
            joint_info = p.getJointInfo(hand_id, -1)  # Use -1 to get information for all joints
            joint_index = next(j[0] for j in joint_info if j[1].decode() == joint_name.encode())
            p.setJointMotorControl2(hand_id, jointIndex=joint_index, controlMode=p.POSITION_CONTROL, targetPosition=joint_angle)

    # Step the PyBullet simulation
    p.stepSimulation()

    # Render the simulation
    p.getCameraImage(800, 600)

    # Display the frame
    cv2.imshow('Hand Gesture Simulation', frame)

    # Break the loop when 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Disconnect from the PyBullet physics server
p.disconnect()

# Release the webcam and close all windows
cap.release()
cv2.destroyAllWindows()


error: GetJointInfo failed.

: 

In [5]:
import pybullet as p
import cv2
import mediapipe as mp
import numpy as np

import cv2
import mediapipe as mp
import pybullet as p
import time
import numpy as np



# Disconnect from the existing PyBullet GUI connection if it exists
try:
    p.disconnect()
except:
    pass

# Connect to the PyBullet physics server with a new GUI connection
physicsClient = p.connect(p.GUI)

# Set gravity in the simulation
p.setGravity(0, 0, -9.8)

# Load a simple hand model into PyBullet
hand_urdf_path = "d://urdf_files/hand/urdf/shadow_hand.urdf"  # Replace with the actual path
hand_id = p.loadURDF(hand_urdf_path, useFixedBase=True)

# Open webcam
cap = cv2.VideoCapture(0)

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()

# Landmark indices mapping
landmarks_mapping = {
    "Wrist": 0,
    "Thumb_Base": 1,
    "Thumb_Knuckle": 2,
    "Thumb_Middle": 3,
    "Thumb_Tip": 4,
    "Index_Knuckle": 5,
    "Index_Middle": 6,
    "Index_Tip": 7,
    "Middle_Knuckle": 9,
    "Middle_Middle": 10,
    "Middle_Tip": 11,
    "Ring_Knuckle": 13,
    "Ring_Middle": 14,
    "Ring_Tip": 15,
    "Pinky_Knuckle": 17,
    "Pinky_Middle": 18,
    "Pinky_Tip": 19,
    "Palm_Base": 0,
    "Palm_Center": 12
}

# Function to map hand landmarks to joint angles for the thumb
def map_thumb(hand_landmarks):
    thumb_base_angle = calculate_joint_angle(hand_landmarks.landmark[landmarks_mapping["Thumb_Base"]])
    thumb_knuckle_angle = calculate_joint_angle(hand_landmarks.landmark[landmarks_mapping["Thumb_Knuckle"]])
    thumb_middle_angle = calculate_joint_angle(hand_landmarks.landmark[landmarks_mapping["Thumb_Middle"]])
    thumb_tip_angle = calculate_joint_angle(hand_landmarks.landmark[landmarks_mapping["Thumb_Tip"]])
    return thumb_base_angle, thumb_knuckle_angle, thumb_middle_angle, thumb_tip_angle

# Function to map hand landmarks to joint angles for the index finger
def map_index(hand_landmarks):
    index_knuckle_angle = calculate_joint_angle(hand_landmarks.landmark[landmarks_mapping["Index_Knuckle"]])
    index_middle_angle = calculate_joint_angle(hand_landmarks.landmark[landmarks_mapping["Index_Middle"]])
    index_tip_angle = calculate_joint_angle(hand_landmarks.landmark[landmarks_mapping["Index_Tip"]])
    return index_knuckle_angle, index_middle_angle, index_tip_angle

# Similar functions for other fingers...

# PyBullet simulation loop
while True:
    # Get hand landmarks from webcam
    _, frame = cap.read()
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = hands.process(frame_rgb)
    hand_landmarks = result.multi_hand_landmarks[0]  # Assuming only one hand

    # Control PyBullet hand based on hand gestures
    thumb_angles = map_thumb(hand_landmarks)
    set_joint_angles("Thumb", thumb_angles)

    index_angles = map_index(hand_landmarks)
    set_joint_angles("Index", index_angles)

    # Similar calls for other fingers...

    # Step the PyBullet simulation
    p.stepSimulation()

    # Display the PyBullet simulation
    # (you may need additional code to visualize the simulated hand)

    # Break the loop on some condition (e.g., press 'q' to quit)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Clean up
cap.release()
cv2.destroyAllWindows()
#calculate angles in 3d space

def calculate_joint_angle(landmark):
    x, y ,z= landmark.x, landmark.y,landmark.z
    return np.arctan2(z,np.sqrt(x**2+y**2))


#define set_joint_angle_function
def set_joint_angles(joint_name, angles):
    joint_index = p.getJointInfo(hand_id, p.getJointIndex(hand_id, joint_name))[0]
    p.setJointMotorControl2(hand_id, joint_index, p.POSITION_CONTROL, targetPosition=angles)
    
    


TypeError: 'NoneType' object is not subscriptable